In [1]:
##Importando os pacotes e módulos necessários
import pandas as pd
import numpy as np
import math

#Para Regressão Linear Simples e Teste F
from scipy import stats
#Para Regressão Linear Múltipla
import statsmodels.api as sm
import econtools
import econtools.metrics as mt

#Pacotes para gráficos
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

#Pacotes para fazer a coleta dos dados armazenados no mesmo diretório e outros pacotes gerais
import os
import pathlib
import glob
from IPython.display import clear_output

##Criando as Funções
def coletar_dados(nome = ""):
    '''
    Função que le os arquivos do Stata
    O arquivo deve estar na mesma pasta do arquivo de Python
    Deixe em branco para ler o arquivo mais recentemente adicionado
    '''
    global df
    #Pegando qual a pasta do arquivo que está sendo usado pra programar
    caminho = pathlib.Path().absolute()

    #checando se o nome foi inserido ou não; caso não, pegar o arquivo .dta mais recente
    if nome == "":
        arquivo = max(glob.glob(f"{str(caminho)}/*.dta"), key=os.path.getctime)
        df = pd.read_stata(arquivo)
        print(f"{arquivo}.dta foi lido com sucesso!")
        return df
    else:
        try:
            arquivo = f"{str(caminho)}/{str(nome)}.dta"
            df = pd.read_stata(arquivo)
            print(f"{nome}.dta foi lido com sucesso!")
            return df
        except: #caso não tenha sido encontrado o arquivo com o nome inserido
            print('''
            Não foi possível achar o arquivo :(\n
            Verifique se seu nome está correto (sem a extensão) e se ele está no mesmo diretório do programa!
            ''')

def Regressão_Simples(Lista_x, Lista_y):
    '''
    Função que calcula as estatísticas de uma regressão simples
    '''
    global Lista_ychapeu_simples, Resíduos_simples
    
    #calculando o número de observações e as médias
    Número_de_Observações = len(Lista_x)
    Média_x = sum(Lista_x)/Número_de_Observações
    Média_y = sum(Lista_y)/Número_de_Observações

    #Calculando os coeficientes do modelo
    B1, B0, R, valor_P, DP = stats.linregress(Lista_x, Lista_y)
    #Gerando os valores previstos
    Lista_ychapeu_simples = [round(B0 + B1 * i,3) for i in Lista_x]
    #Calculando os Resíduos
    Resíduos_simples = [(j - k) for j,k in zip(Lista_y, Lista_ychapeu_simples)]
    
    #Calculando R-quadrados e a Soma dos Quadrados das Variáveis
    R_quadrado = round(R**2,5)
    SQTx = sum([(i - Média_x)**2 for i in Lista_x])
    SQTy = sum([(i - Média_y)**2 for i in Lista_y])
    SQEy = sum([(i - Média_y)**2 for i in Lista_ychapeu_simples])
    SQR = sum([i**2 for i in Resíduos_simples]) 

    #Calculando a Variância da Regressão e dos  Coeficientes
    VarianciaReg = SQR/(Número_de_Observações - 2)
    EPR = math.sqrt(VarianciaReg)
    VarB1 = VarianciaReg/SQTx
    VarB0 = (VarianciaReg * sum([i**2 for i in Lista_x]))/(Número_de_Observações * SQTx)
    
    #Calculando da estatistica t com intervalo de confiança de 95%  (p/ gerar os intervalos de confiança dos estimadores)
    Estatistica_t_Critica = stats.t.ppf(0.95, Número_de_Observações - 2)

    #Calculando os estimadores do limite inferior e superior
    B1_inferior = B1 - math.sqrt(VarB1) * Estatistica_t_Critica
    B1_superior = B1 + math.sqrt(VarB1) * Estatistica_t_Critica
    B0_inferior = B0 - math.sqrt(VarB0) * Estatistica_t_Critica
    B0_superior = B0 + math.sqrt(VarB0) * Estatistica_t_Critica
    
    #Gerando o Relatório
    Relatório = f'''
    Número de Observações = {Número_de_Observações}\n
    B0 = {round(B0,5)}\t B1 = {round(B1,5)}\t R-quadrado = {R_quadrado}\n
    Estimador da Variância = {round(VarianciaReg,5)}\t Erro Padrão da Regressão = {round(EPR,5)}\n
    Variância de B1 = {round(VarB1,5)}\t Variância de B0 = {round(VarB0,5)}\n
    Intervalo de Confiança de 95% para B1 (Inferior; B1; Superior): {round(B1_inferior,4)}; {round(B1,4)}; {round(B1_superior,4)}\n
    Intervalo de Confiança de 95% para B0 (Inferior; B0; Superior): {round(B0_inferior,4)}; {round(B0,4)}; {round(B0_superior,4)}\n
    Para ver os valores previstos, basta chamar a variável 'Lista_ychapeu_simples'\n
    Para ver os resíduos, chame a variável 'Resíduos_simples'
    '''
    print (Relatório)
    
    ##Criando um gráfico
    #Deixando o estilo bonitinho
    sns.set_style(style="white")

    #Criando o objeto gráfico
    Grafico = sns.regplot(x = Lista_x, y = Lista_y, scatter_kws={"color": "black"}, line_kws={"color": "red"})
    Grafico.set_title("Resultado da Regressão",fontsize = 11)
    plt.show()

def Regressão_Múltipla(x,y,constante="S"):
    global Resultado, Lista_ychapeu, Resíduos, SQR, EPR

    #adicionando uma constante ao modelo de Ordinary Least Squares(OLS)
    if constante == "S":
        X = sm.add_constant(x)
    else:
        X = x
    #Criando o Modelo
    Modelo = sm.OLS(y,X)
    Resultado = Modelo.fit()
    
    Lista_ychapeu = Resultado.predict()
    Resíduos = y - Lista_ychapeu

    #Calculando o Erro Padrão da Regressão (EPR)
    SQR =sum([i**2 for i in Resíduos])
    Número_de_Observações = len(y)
    GL = Número_de_Observações - len(Resultado.params)
    VarianciaReg = SQR/GL
    EPR = math.sqrt(SQR/GL)
    
    ##Printando o Resultado
    #print('Parâmetros:\n', Resultado.params) #O primeiro resultado equivale ao intercepto
    #print('\nDesvios Padrões:\n', Resultado.bse)
    #print('Valores Previstos: ', Resultado.predict())
    #print('\nR2:', Resultado.rsquared)
    print(f"O erro padrão da regressão é {round(EPR,5)} e a SQR é {round(SQR,5)}\n")
    print(Resultado.summary())

    print("\nPara ver os valores previstos ou os resídudos, basta chamar 'Lista_ychapeu' e 'Resíduos'.")
    print("Os resultados do modelo podem ser obtidos através de métodos usando a variável 'Resultado'.")
  
def Teste_F_Restrições_de_Exclusão(x, y, Restrições, Nivel_de_Significância = 0.05):
    ##Definindo as variáveis de cada modelo
    ModeloIrrestrito = list(x)
    ModeloRestrito = []
    Restrições = list(Restrições)

    Numero_de_Observações = len(y)
    GL_ir = Numero_de_Observações - (len(ModeloIrrestrito) + 1)
    GL_r = len(Restrições)

    for i in ModeloIrrestrito:
        if i not in Restrições:
            ModeloRestrito.append(i)

    ##Fazendo as regressões de cada modelo
    Regressão_Múltipla(x, y)
    SQR_ir = SQR
    VarianciaReg_ir = EPR**2

    Regressão_Múltipla(df[ModeloRestrito], y)
    SQR_r = SQR

    #Limpando a tela
    clear_output()
    
    ##Calculando F
    F = (SQR_r - SQR_ir)/(len(Restrições)*VarianciaReg_ir)

    ##Calculando o P-valor de F
    P_valor = stats.f.sf(F,GL_r,GL_ir)

    if Nivel_de_Significância > P_valor:
        print(f"O valor de F é {round(F,3)} e seu p-valor é {round(P_valor,7)}. Portanto, rejeita-se Ho a um nível de significância de {Nivel_de_Significância*100}%.")
    else:
        print(f"O valor de F é {round(F,3)} e seu p-valor é {round(P_valor,7)}. Portanto, não se rejeita Ho a um nível de significância de {Nivel_de_Significância*100}%.")
     

In [3]:
coletar_dados('gpa1')

gpa1.dta foi lido com sucesso!


,age,soph,junior,senior,senior5,male,campus,business,engineer,colGPA,...,greek,car,siblings,bgfriend,clubs,skipped,alcohol,gradMI,fathcoll,mothcoll
0,21,0,0,1,0,0,0,1,0,3.0,...,0,1,1,0,0,2.0,1.00,1,0,0
1,21,0,0,1,0,0,0,1,0,3.4,...,0,1,0,1,1,0.0,1.00,1,1,1
2,20,0,1,0,0,0,0,1,0,3.0,...,0,1,1,0,1,0.0,1.00,1,1,1
3,19,1,0,0,0,1,1,1,0,3.5,...,0,0,1,0,0,0.0,0.00,0,0,0
4,20,0,1,0,0,0,0,1,0,3.6,...,0,1,1,1,0,0.0,1.50,1,1,0
5,20,0,0,1,0,1,1,1,0,3.0,...,0,1,1,0,0,0.0,0.00,0,1,0
6,22,0,0,0,1,0,0,1,0,2.7,...,1,1,1,0,1,0.0,2.00,1,0,1
7,22,0,0,0,1,0,0,0,0,2.7,...,0,0,1,1,0,3.0,3.00,1,1,1
8,22,0,0,0,1,0,0,0,0,2.7,...,0,1,1,1,1,2.0,2.50,1,1,1
9,19,1,0,0,0,0,0,1,0,3.8,...,0,0,1,0,1,0.5,0.75,1,0,1


In [9]:
#Fazendo uso do pacote econtools.metrics
results = mt.reg(df, 'colGPA', ['skipped', 'hsGPA','ACT'], addcons=True)
# Print a nice summary of the regression results (a string)
print(results)

# Print DataFrame w/ betas, se's, t-stats, etc.
#print(results.summary)

# Print only betas
print(results.beta)

# Print std. err. for `educ` coefficient
#print(results.se['skipped'])

# Print full variance-covariance matrix
#print(results.vce)

Dependent variable:	colGPA
N:			141
R-squared:		0.2336
Estimation method:	OLS
VCE method:		Standard (Homosk.)
         coeff    se      t   p>t CI_low CI_high
skipped -0.083 0.026 -3.197 0.002 -0.135  -0.032
hsGPA    0.412 0.094  4.396 0.000  0.227   0.597
ACT      0.015 0.011  1.393 0.166 -0.006   0.036
_cons    1.390 0.332  4.191 0.000  0.734   2.045

skipped   -0.083113
hsGPA      0.411816
ACT        0.014720
_cons      1.389554
dtype: float64


In [14]:
#Teste F
# Test for joint significance (mesma coisa de usar a função Teste_F_Restrições de Exclusão)
F1, p_valueF1 = results.Ftest(['hsGPA', 'ACT'])
print(F1,p_valueF1)
# Test for equality
F2, p_valueF2 = results.Ftest(['ACT', 'skipped'], equal=True)
print(F2,p_valueF2)

14.751277575369388 1.5777094370950095e-06
10.9555915098771 0.001192820631725544


## Intervalo de Confiança de E(y|x)

In [4]:
#Suponha que queremos testar para hsGPA = 3, ACT = 22 e skipped = 3
df['hsGPA0'] = df['hsGPA'] - 3
df['ACT0'] = df['ACT'] - 22
df['skipped0'] = df['skipped'] - 3

In [6]:
Regressão_Múltipla(df[['hsGPA0','ACT0','skipped0']],df['colGPA'])
#estamos interessados nos valores da constante! note que todas as outras estimativas e R2 sao iguais

O erro padrão da regressão é 0.32949 e a SQR é 14.87297

                            OLS Regression Results                            
Dep. Variable:                 colGPA   R-squared:                       0.234
Model:                            OLS   Adj. R-squared:                  0.217
Method:                 Least Squares   F-statistic:                     13.92
Date:                Wed, 28 Oct 2020   Prob (F-statistic):           5.65e-08
Time:                        10:20:18   Log-Likelihood:                -41.501
No. Observations:                 141   AIC:                             91.00
Df Residuals:                     137   BIC:                             102.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------